In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from interpretableai import iai
import pandas as pd
from sklearn.metrics import r2_score

1f302ee744f0c5b56a7bdfadaaee583e4266588ca0b8a032e72e0fc0ea74e74d


In [4]:
data = pd.read_csv("features.csv")
open_cols = [col for col in data.columns if "Open" in col]
coins = [col.split("_")[1] for col in open_cols]

df_train = pd.read_csv("features_train.csv")
df_test1 = pd.read_csv("features_val.csv")
df_test2 = pd.read_csv("features_test.csv")

In [5]:
randomState = 42

In [ ]:
tree_learners = []
for coin in coins:
    coin_cols = [col for col in df_train.columns if coin in col]
    df_coin = df_train[coin_cols]
    nrows = df_coin.shape[0]
    train_X = df_coin.drop(columns="Log_RET_{}".format(coin))[65:nrows-1]
    train_y = df_coin["Log_RET_{}".format(coin)][66:nrows]

    df_coin_test = df_test1[coin_cols]
    test_X = df_coin_test.drop(columns="Log_RET_{}".format(coin))[0:nrows-1]
    test_y = df_coin_test["Log_RET_{}".format(coin)][1:nrows]

    grid = iai.GridSearch(iai.OptimalTreeRegressor(random_seed=randomState, regression_features={'All'}),max_depth=[5],regression_lambda=[0.05])
    grid.fit(train_X, train_y)
    #grid.fit_cv(train_X, train_y)

    #get tree
    grid.write_html("results/" + coin + "_tree.html")
    bestLearner = grid.get_learner()
    tree_learners.append(bestLearner)

    train_y_hat = bestLearner.predict(train_X)
    cvAcc = bestLearner.score(train_X, train_y, criterion="mse")
    cvR2 = r2_score(train_y, train_y_hat)
    print("Train MSE for {}: {}".format(coin, cvAcc))
    print("Train R2 for {}: {}".format(coin, cvR2))